# Reverse-Mode Automatic Differentiation Engine
**Author:** Rukmini Nazre

This notebook demonstrates a custom-built Automatic Differentiation (AD) engine implemented from scratch in Python and NumPy. It supports reverse-mode differentiation (backpropagation) through a dynamic computational graph, similar to PyTorch's `autograd`.

**Key Features:**
* Dynamic graph construction using `Var` objects.
* Support for scalar and tensor operations (`matmul`, `solve`, `logdet`, etc.).
* Numerical validation against standard finite-difference approximations.

In [1]:
import numpy as np
import autodiff as ad  # import custom library
from autodiff import Var 

# random seed for reproducibility
np.random.seed(42)

## Demo 1: Simple Scalar Arithmetic
First, we validate the engine on a simple function:
$$z = x \cdot y + x$$

We expect the gradients to be:
* $\frac{\partial z}{\partial x} = y + 1$
* $\frac{\partial z}{\partial y} = x$

In [ ]:
# 1. define inputs
x_val = 2.0
y_val = 3.0

# 2. define the computation function
def simple_math(x, y):
    # z = x * y + x
    return x * y + x 

# 3. compute gradients using the autodiff engine
# ad.grad takes a function and returns a new function that outputs gradients
grad_fn = ad.grad(simple_math)
grads = grad_fn(x_val, y_val)

print(f"Inputs: x={x_val}, y={y_val}")
print(f"Computed Gradient dx: {grads[0]} (Expected: {y_val + 1})")
print(f"Computed Gradient dy: {grads[1]} (Expected: {x_val})")

Inputs: x=2.0, y=3.0
Computed Gradient dx: 4.0 (Expected: 4.0)
Computed Gradient dy: 2.0 (Expected: 2.0)


## Demo 2: Multivariate Gaussian Log-Likelihood
This example demonstrates the engine's ability to handle complex Linear Algebra operations. We compute the gradients for the **Negative Log-Likelihood** of a Multivariate Gaussian distribution with respect to the covariance matrix $\Sigma$.

The operations involved include:
* Matrix Multiplication (`@`)
* Linear Solve (`np.linalg.solve`)
* Log Determinant (`np.linalg.slogdet`)
* Summation and Trace operations

In [3]:
# data dimensions
N = 100  # number of samples
D = 5    # dimension of features

# generate synthetic data (centering data to mean 0 for simplicity)
X_np = np.random.randn(N, D)

# generate a valid positive-definite covariance matrix Sigma
A = np.random.randn(D, D)
Sigma_np = A @ A.T + 0.1 * np.eye(D)  # ensure invertibility

print(f"Data Shape: {X_np.shape}")
print(f"Covariance Matrix Shape: {Sigma_np.shape}")

Data Shape: (100, 5)
Covariance Matrix Shape: (5, 5)


In [4]:
# --- 1. Define the Loss Function using AutoDiff Operators ---
def gaussian_log_likelihood_loss(Sigma):
    # Loss = (N/2)*logdet(Sigma) + 0.5 * sum( x.T @ inv(Sigma) @ x )
    
    # Term 1: Log Determinant
    term1 = ad.mul(ad.Var(N / 2), ad.logdet(Sigma))
    
    # Term 2: Mahalanobis Distance (using solve for stability instead of explicit inv)
    # We want sum(X_i^T @ Sigma^-1 @ X_i). 
    # Efficient trick: sum(X * solve(Sigma, X.T).T) equivalent to trace/sum logic
    
    # Compute Sigma^-1 @ X.T (Shape: D x N)
    X_var = ad.Var(X_np.T) # Transpose to (D, N) for solve
    inv_sigma_x = ad.solve(Sigma, X_var) 
    
    # Compute quadratic term: 0.5 * sum(X_var * inv_sigma_x) 
    # Element-wise multiply and sum acts like the trace of the product
    product = X_var * inv_sigma_x
    term2 = ad.mul(ad.Var(0.5), ad._sum(product))
    
    return term1 + term2

# --- 2. Numerical Gradient Helper (Finite Difference) ---
def numerical_gradient(f, x, eps=1e-6):
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index
        orig_val = x[idx]
        
        # f(x + eps)
        x[idx] = orig_val + eps
        f_plus = f(x)
        
        # f(x - eps)
        x[idx] = orig_val - eps
        f_minus = f(x)
        
        # Finite difference
        grad[idx] = (f_plus - f_minus) / (2 * eps)
        
        x[idx] = orig_val # Restore
        it.iternext()
        
    return grad

# Wrapper for numerical check (only needs numpy ops)
def numpy_loss_wrapper(Sigma_val):
    sign, logdet = np.linalg.slogdet(Sigma_val)
    term1 = (N / 2) * logdet
    
    # Manual inverse for numpy check
    inv_sigma = np.linalg.inv(Sigma_val)
    # term2 = 0.5 * sum( (x @ inv_sigma @ x.T) )
    term2 = 0.5 * np.sum(X_np @ inv_sigma * X_np) 
    
    return term1 + term2

In [5]:
print("Running Backpropagation...")

# 1. Compute Gradient using AutoDiff Engine
grad_fn = ad.grad(gaussian_log_likelihood_loss)
autodiff_grads = grad_fn(Sigma_np)
grad_sigma_autodiff = autodiff_grads[0]

# 2. Compute Numerical Gradient for Verification
print("Computing Numerical Gradient (this may take a moment)...")
grad_sigma_numerical = numerical_gradient(numpy_loss_wrapper, Sigma_np.copy())

# 3. Compare Results
diff = np.linalg.norm(grad_sigma_autodiff - grad_sigma_numerical)
denom = np.linalg.norm(grad_sigma_autodiff) + np.linalg.norm(grad_sigma_numerical)
rel_error = diff / denom

print("-" * 30)
print(f"Gradient Norm (AutoDiff): {np.linalg.norm(grad_sigma_autodiff):.6f}")
print(f"Gradient Norm (Numerical): {np.linalg.norm(grad_sigma_numerical):.6f}")
print(f"Relative Error: {rel_error:.2e}")
print("-" * 30)

if rel_error < 1e-5:
    print("SUCCESS: AutoDiff gradients match numerical approximation!")
else:
    print("WARNING: Gradients differ. Check implementation.")

Running Backpropagation...
Computing Numerical Gradient (this may take a moment)...
------------------------------
Gradient Norm (AutoDiff): 134.882423
Gradient Norm (Numerical): 134.882423
Relative Error: 4.01e-10
------------------------------
SUCCESS: AutoDiff gradients match numerical approximation!
